# 93_fig3_persistent_vs_temporary

Figure 3: persistent regime switch vs temporary ξ shock with matched inflation impact (commitment or discretion).

In [ ]:
import os, sys, json, numpy as np, torch, matplotlib.pyplot as plt
import pathlib

def _find_project_root():
    here = pathlib.Path.cwd().resolve()
    for p in [here, *here.parents]:
        if (p / "src").is_dir():
            return p
    # Common Google Colab clone location
    cand = pathlib.Path("/content/econml")
    if (cand / "src").is_dir():
        return cand
    raise RuntimeError("Could not find project root containing src/. If on Colab, clone repo to /content/econml.")

PROJECT_ROOT = _find_project_root()
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

from src.config import ModelParams
from src.io_utils import load_json, load_npz, load_torch, load_selected_run, find_latest_run_dir
from src.deqn import PolicyNetwork

ART = str(PROJECT_ROOT / "artifacts" / "runs")

def get_run(policy: str) -> str:
    rd = load_selected_run(ART, policy)
    if rd is None:
        rd = find_latest_run_dir(ART, policy)
    if rd is None:
        raise RuntimeError(f"No runs found for policy={policy} under {ART}")
    return rd

def _parse_dtype(s: str):
    if s is None:
        return torch.float32
    if isinstance(s, torch.dtype):
        return s
    s = str(s)
    if "float64" in s:
        return torch.float64
    if "float32" in s:
        return torch.float32
    if "bfloat16" in s:
        return torch.bfloat16
    return torch.float32

def load_params_from_run(run_dir: str, *, device="cpu"):
    cfg = load_json(os.path.join(run_dir, "config.json"))
    p = cfg.get("params", {})
    dtype = _parse_dtype(p.get("dtype"))
    dev = device if device is not None else p.get("device","cpu")
    keep = {k:v for k,v in p.items() if k in ModelParams.__dataclass_fields__}
    keep["device"] = dev
    keep["dtype"] = dtype
    return ModelParams(**keep).to_torch()

def load_net_from_run(run_dir: str, d_in: int, d_out: int):
    cfg = load_json(os.path.join(run_dir, "config.json"))
    tc = cfg.get("train_cfg", {})
    hidden = tuple(tc.get("hidden_layers", (512,512)))
    activation = tc.get("activation", "selu")
    p = cfg.get("params", {})
    net_dtype = _parse_dtype(p.get("dtype"))
    net = PolicyNetwork(d_in, d_out, hidden=hidden, activation=activation).to(device="cpu", dtype=net_dtype)
    state = load_torch(os.path.join(run_dir, "weights.pt"), map_location="cpu")
    # state is usually a plain state_dict
    if isinstance(state, dict) and "state_dict" in state:
        state = state["state_dict"]
    net.load_state_dict(state)
    net.eval()
    return net

# --- paper reporting helpers ---
ann = lambda x: 400.0*x  # annualized percent (quarterly -> annual)


In [ ]:
from src.steady_states import solve_commitment_sss_from_policy_switching, solve_discretion_sss_from_policy_switching
from src.experiments import DeterministicPathSpec, simulate_deterministic_path, calibrate_xi_jump_to_match_pi_impact

POLICY = "commitment"  # or "discretion"
run_dir = get_run(POLICY)
params = load_params_from_run(run_dir)

if POLICY == "commitment":
    net = load_net_from_run(run_dir, 7, 13)
    sss = solve_commitment_sss_from_policy_switching(params, net)
    x0 = torch.tensor([[float(sss.by_regime[0]["Delta_prev"]), float(sss.by_regime[0]["logA"]), float(sss.by_regime[0]["loggtilde"]), float(sss.by_regime[0]["xi"]), 0.0,
                        float(sss.by_regime[0]["vartheta_prev"]), float(sss.by_regime[0]["varrho_prev"])]], dtype=torch.float32)
else:
    net = load_net_from_run(run_dir, 5, 11)
    sss = solve_discretion_sss_from_policy_switching(params, net)
    x0 = torch.tensor([[float(sss.by_regime[0]["Delta_prev"]), float(sss.by_regime[0]["logA"]), float(sss.by_regime[0]["loggtilde"]), float(sss.by_regime[0]["xi"]), 0.0]], dtype=torch.float32)

T = 40
spec_switch = DeterministicPathSpec(T=T, epsA=0.0, epsg=0.0, epst=0.0, regime_path=[0] + [1] * T)
path_switch = simulate_deterministic_path(params, POLICY, net, x0=x0, spec=spec_switch, compute_implied_i=True)

# With regime_path=[0]+[1]*T, the forced switch applies to x_{t+1}; impact appears at index 1.
target_pi0 = float(path_switch["pi"][1].mean())

xi_jump = calibrate_xi_jump_to_match_pi_impact(
    params, POLICY, net, x0=x0, target_pi0=target_pi0, horizon_T=1, regime_path=[0, 0]
)
x0_xi = x0.clone(); x0_xi[:, 3] += xi_jump
spec_temp = DeterministicPathSpec(T=T, epsA=0.0, epsg=0.0, epst=0.0, regime_path=[0] * (T + 1))
path_temp = simulate_deterministic_path(params, POLICY, net, x0=x0_xi, spec=spec_temp, compute_implied_i=True)

# Align timelines so both series start at impact period (plot t=0).
pi_switch = path_switch["pi"][1:, 0]   # length T
pi_temp = path_temp["pi"][:T, 0]       # length T

# Price levels normalized to 1 at impact date.
def price_level_from_pi(pi_series: np.ndarray) -> np.ndarray:
    out = np.ones(len(pi_series) + 1, dtype=np.float64)
    if len(pi_series) > 0:
        out[1:] = np.cumprod(1.0 + np.asarray(pi_series, dtype=np.float64))
    return out

P_switch = price_level_from_pi(pi_switch)
P_temp = price_level_from_pi(pi_temp)

t = np.arange(T)
tP = np.arange(T + 1)

fig, ax = plt.subplots(1, 2, figsize=(11, 4))

ax[0].plot(t, ann(pi_switch), label="Persistent (regime switch)")
ax[0].plot(t, ann(pi_temp), label="Temporary xi shock", linestyle="--")
ax[0].axhline(0.0, linewidth=1)
ax[0].set_title("Figure 3a: Inflation paths (matched impact)")
ax[0].set_xlabel("t")
ax[0].set_ylabel("Annualized percent")
ax[0].legend()

ax[1].plot(tP, P_switch, label="Persistent (regime switch)")
ax[1].plot(tP, P_temp, label="Temporary xi shock", linestyle="--")
ax[1].set_title("Figure 3b: Price level")
ax[1].set_xlabel("t")
ax[1].set_ylabel("Index (normalized at impact)")
ax[1].legend()

plt.tight_layout()
plt.show()

print("Matched impact (aligned t=0) target_pi0:", target_pi0, "xi_jump:", xi_jump)
